Article from Philippe:
https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html


In [1]:
# load biogen data
import pandas as pd

biogen_data=pd.read_csv("ADME_public_set_3521.csv")
print(biogen_data.columns)

#load bioavailabity data
bio_avail_data = pd.read_csv("11095_2013_1222_MOESM2_ESM.csv",sep=";")
print(bio_avail_data.columns)



Index(['Internal ID', 'Vendor ID', 'SMILES', 'CollectionName',
       'LOG HLM_CLint (mL/min/kg)', 'LOG MDR1-MDCK ER (B-A/A-B)',
       'LOG SOLUBILITY PH 6.8 (ug/mL)',
       'LOG PLASMA PROTEIN BINDING (HUMAN) (% unbound)',
       'LOG PLASMA PROTEIN BINDING (RAT) (% unbound)',
       'LOG RLM_CLint (mL/min/kg)'],
      dtype='object')
Index(['No', 'Name', 'Updated SMILES', '%F', 'logK(%F)', 'Category', 'Source'], dtype='object')


use feed-forward NN using pytorch.
train on solubility from dataset 1 first.
then optimize for bioavailability

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from deepchem.feat import RDKitDescriptors

#### DATA PREPARATION ####

# Function to generate features from SMILES strings using RDKit descriptors
def generate_features(smiles_list):
    featurizer = RDKitDescriptors()
    features = featurizer.featurize(smiles_list)
    # Drop features containing invalid values
    features = features[:, ~np.isnan(features).any(axis=0)]
    return features

#remove nan values from data
biogen_data = biogen_data.dropna(subset=['LOG SOLUBILITY PH 6.8 (ug/mL)'])

#get x and y data (x is the molecular descriptors, y is the solubility)
y_data = biogen_data["LOG SOLUBILITY PH 6.8 (ug/mL)"]

print(len(y_data))

# Generate features from SMILES data (get smiles from df)
smiles = biogen_data["SMILES"]
X_data = generate_features(smiles)

#split data into training and validation using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)
# Convert y pandas Series to NumPy array
y_train = y_train.values.astype(np.float32)
y_test = y_test.values.astype(np.float32)

#convert data to pytorch tensors (like numpy arrays but for pytorch)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Reshape the target tensor to match the shape of the output tensor
y_train_tensor = y_train_tensor.view(-1, 1)
# Reshape the target tensor to match the shape of the output tensor
y_test_tensor = y_test_tensor.view(-1, 1)


In [14]:
y_train_tensor

tensor([[ 3.0866,  2.5215, -2.4641,  ...,  0.0000,  0.0000,  0.8025],
        [ 3.2671,  2.4088, -2.1844,  ...,  0.0000,  0.0000,  0.9169],
        [ 3.0378,  2.2070, -2.3582,  ...,  0.0000,  0.0000,  0.7843],
        ...,
        [ 2.9562,  2.1637, -2.0880,  ...,  0.0000,  0.0000,  0.6843],
        [ 2.1544,  2.1185, -2.2393,  ...,  0.0000,  0.0000,  0.7248],
        [ 2.9430,  2.0381, -1.9899,  ...,  0.0000,  0.0000,  0.7946]])

In [15]:
#### CREATING MODEL ####

# Define the neural network architecture
class SolubilityPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SolubilityPredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define hyperparameters
input_dim = X_data.shape[1]  # Number of molecular descriptors
hidden_dim = 64  # Adjust to get best results
output_dim = 1
learning_rate = 0.001
num_epochs = 50
batch_size = 32

#create dataloader for batch training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#initialize model, loss function (nn.MSELoss) and optimizer
model = SolubilityPredictor(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss}")


AssertionError: Size mismatch between tensors

In [9]:

# Evaluation
with torch.no_grad():
    val_outputs = model(X_test_tensor)
    val_loss = criterion(val_outputs, y_test_tensor)
    print(f"Validation Loss: {val_loss.item()}")

Validation Loss: nan


c:\Users\nina\anaconda3\envs\ai_chem\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([705])) that is different to the input size (torch.Size([705, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
